In [1]:
import pandas as pd
import random
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
from sentence_transformers import CrossEncoder
import numpy as np

2025-05-04 17:44:02.135753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746380642.314299      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746380642.365410      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import os

os.environ['KAGGLE_USERNAME'] = '' # YOUR USERNAME
os.environ['KAGGLE_KEY'] = ''  # YOUR API KEY

In [6]:
ds_train = load_dataset("FreedomIntelligence/RAG-Instruct", split="train[:80%]")
ds_test = load_dataset("FreedomIntelligence/RAG-Instruct", split="train[80%:]")

README.md:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

rag_instruct.json:   0%|          | 0.00/296M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40541 [00:00<?, ? examples/s]

In [7]:
docs_train = [d for s in ds_train['documents'] for d in s]
questions_train = [q for q in ds_train['question']]

docs_test = [d for s in ds_test['documents'] for d in s]
questions_test = [q for q in ds_test['question']]

#### Sentence Transformer

Please refer to https://www.sbert.net/docs/sentence_transformer/pretrained_models.html for the models.

In [20]:
st_model = 'multi-qa-mpnet-base-cos-v1'

model = SentenceTransformer(st_model)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
docs_embeddings_train = model.encode(docs_train, batch_size=64, convert_to_tensor=True)

docs_embeddings_test = model.encode(docs_test, batch_size=64, convert_to_tensor=True)

Batches:   0%|          | 0/5068 [00:00<?, ?it/s]

Batches:   0%|          | 0/1267 [00:00<?, ?it/s]

In [22]:
questions_embeddings_train = model.encode(questions_train, batch_size=64, convert_to_tensor=True)

questions_embeddings_test = model.encode(questions_test, batch_size=64, convert_to_tensor=True)

Batches:   0%|          | 0/507 [00:00<?, ?it/s]

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

In [23]:
docs_embeddings_train = docs_embeddings_train.cpu().numpy()
questions_embeddings_train = questions_embeddings_train.cpu().numpy()

docs_embeddings_test = docs_embeddings_test.cpu().numpy()
questions_embeddings_test = questions_embeddings_test.cpu().numpy()

In [24]:
np.save(f"/kaggle/working/st_docs_embeddings_train-{st_model}.npy", docs_embeddings_train)
np.save(f"/kaggle/working/st_docs_embeddings_test-{st_model}.npy", docs_embeddings_test)

In [26]:
np.save(f"/kaggle/working/st_qsts_embeddings_train-{st_model}.npy", questions_embeddings_train)
np.save(f"/kaggle/working/st_qsts_embeddings_test-{st_model}.npy", questions_embeddings_test)

In [27]:
import json

metadata_path = "/kaggle/working/dataset-metadata.json"

with open(metadata_path, "r") as f:
    metadata = json.load(f)

username = "chferra"

title = "multi qa mpnet base cos v1" # NAME OF THE DATASET, PLS USE THE NAME OF THE MODEL WITH SPACES INSTEAD OF -
slug = "multi-qa-mpnet-base-cos-v1" # SAME AS THE TITLE BUT WITH - INSTEAD

metadata["title"] = title
metadata["id"] = f"{username}/{slug}"
metadata["licenses"] = [{"name": "CC0-1.0"}]  # not important

with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

✅ Metadata updated.


#### Cross encoder

In [11]:
xenc_model = 'stsb-distilroberta-base'

xmodel = CrossEncoder(f'cross-encoder/{xenc_model}')

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [12]:
def build_xenc_input(questions, docs):
    xinputs = []
    for i in range(len(questions)):
        for j in range(10):
            xinputs.append((questions[i], docs[i * 10 + j]))
    return xinputs

In [13]:
xinputs_train = build_xenc_input(questions_train, docs_train)
xinputs_test = build_xenc_input(questions_test, docs_test)

In [14]:
def xenc_predict(xinputs, questions):
    scores = xmodel.predict(xinputs)
    grouped_scores = [scores[i*10:(i+1)*10] for i in range(len(questions))]
    per_q_scores = []
    for i in range(len(grouped_scores)):
        scores = grouped_scores[i]
        score_index_pairs = [(score, j) for j, score in enumerate(scores)]
        score_index_pairs.sort(reverse=True)
        per_q_scores.append(score_index_pairs)
    return per_q_scores

In [15]:
per_q_scores_train = xenc_predict(xinputs_train, questions_train)
per_q_scores_test = xenc_predict(xinputs_test, questions_test)

Batches:   0%|          | 0/10136 [00:00<?, ?it/s]

Batches:   0%|          | 0/2534 [00:00<?, ?it/s]

In [16]:
np.save(f"/kaggle/working/xenc_scores_train-{xenc_model}.npy", per_q_scores_train)
np.save(f"/kaggle/working/xenc_scores_test-{xenc_model}.npy", per_q_scores_test)